# Set up an efficient and accurate simulation

```{note}
TODO: write notebook describing how to use `dt` and integration methods in Kernels to control numerical accuracy. Use [Michaels tutorial](https://github.com/Parcels-code/10year-anniversary-session2/blob/main/advection_and_windage.ipynb) as a starting point.
```